## Alpaca Trading API Overview

In [1]:
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
from alpaca.trading.requests import GetAssetsRequest
from alpaca.data import StockHistoricalDataClient
from alpaca.data.requests import StockLatestQuoteRequest
import pandas as pd

Note: log on to alpaca at https://alpaca.markets/ to get key and secret key

In [2]:
KEY = "PKXDK3VJZBL7PRXS3F28"
SECRET_KEY = "bwho7ZbVcQKIaYgYcLI67c0Sibxj3ckOmnxmwjNv"
trading_client = TradingClient(KEY, SECRET_KEY, paper=True)

### View account information

In [3]:
# Get our account information.
account = trading_client.get_account()
account

{   'account_blocked': False,
    'account_number': 'PA326D2U1D75',
    'accrued_fees': '0',
    'buying_power': '200008.08',
    'cash': '100004.04',
    'created_at': datetime.datetime(2023, 12, 21, 14, 56, 37, 240661, tzinfo=datetime.timezone.utc),
    'crypto_status': <AccountStatus.ACTIVE: 'ACTIVE'>,
    'currency': 'USD',
    'daytrade_count': 2,
    'daytrading_buying_power': '0',
    'equity': '100004.04',
    'id': UUID('22f9a871-da4e-45c6-a8d7-e034a2f61fc1'),
    'initial_margin': '0',
    'last_equity': '100000',
    'last_maintenance_margin': '0',
    'long_market_value': '0',
    'maintenance_margin': '0',
    'multiplier': '2',
    'non_marginable_buying_power': '100000',
    'pattern_day_trader': False,
    'pending_transfer_in': '0',
    'pending_transfer_out': None,
    'portfolio_value': '100004.04',
    'regt_buying_power': '200008.08',
    'short_market_value': '0',
    'shorting_enabled': True,
    'sma': '0',
    'status': <AccountStatus.ACTIVE: 'ACTIVE'>,
    'tr

In [4]:
# Check if our account is restricted from trading.
if account.trading_blocked:
    print('Account is currently restricted from trading.')

In [5]:
# Check how much money we can use to open new positions and current value
print(f'Available buying power:\t${float(account.buying_power):,.2f}')
print(f'Portfolio value:\t${float(account.portfolio_value):,.2f}')

Available buying power:	$200,008.08
Portfolio value:	$100,004.04


In [6]:
# Check our current balance vs. our balance at the last market close
balance_change = float(account.equity) - float(account.last_equity)
print(f'Today\'s portfolio balance change: ${balance_change:,.2f}')


Today's portfolio balance change: $4.04


### Asset and Current Pricing Information from Alpaca

In [7]:
# get information about a particular asset
SYMBOL = 'SPY'
asset_info = trading_client.get_asset(SYMBOL)
if asset_info.tradable:
    print(f'We can trade {SYMBOL}.')
asset_info

We can trade SPY.


{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'easy_to_borrow': True,
    'exchange': <AssetExchange.ARCA: 'ARCA'>,
    'fractionable': True,
    'id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'maintenance_margin_requirement': 30.0,
    'marginable': True,
    'min_order_size': None,
    'min_trade_increment': None,
    'name': 'SPDR S&P 500 ETF Trust',
    'price_increment': None,
    'shortable': True,
    'status': <AssetStatus.ACTIVE: 'active'>,
    'symbol': 'SPY',
    'tradable': True}

In [8]:
# get list of equities
from alpaca.trading.enums import AssetClass
search_params = GetAssetsRequest(asset_class=AssetClass.US_EQUITY)
assets = trading_client.get_all_assets(search_params)
assets[0]

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'easy_to_borrow': False,
    'exchange': <AssetExchange.OTC: 'OTC'>,
    'fractionable': False,
    'id': UUID('1bdc1fcc-7d8c-43bd-925a-8d9338ea7040'),
    'maintenance_margin_requirement': 100.0,
    'marginable': False,
    'min_order_size': None,
    'min_trade_increment': None,
    'name': 'Essential Energy Services Ltd. Common Shares (Canada)',
    'price_increment': None,
    'shortable': False,
    'status': <AssetStatus.INACTIVE: 'inactive'>,
    'symbol': 'EEYUF',
    'tradable': False}

In [9]:
# get tradeable and shortable stocks
assets = [
    x for x in assets 
    if (x.asset_class[:]=='us_equity') 
    and (x.status[:]=='active')
]
symbols   = [x.symbol for x in assets]
tradable  = [x.tradable for x in assets]
shortable = [x.shortable for x in assets]

# df = pd.DataFrame([tradable,shortable])
df = pd.DataFrame(list(zip(tradable, shortable)), columns=['tradable','shortable'], index=symbols)
df.head()

,tradable,shortable
DRTTF,False,False
HCDPQ,False,False
HYW,False,True
CKHUY,False,True
CTNT,False,False


In [10]:
# get quotes for set of symbols
SYMBOL_LIST = ['SPY','AAPL','IBM']
df = df.loc[SYMBOL_LIST]

# pull data from alpaca
data_client = StockHistoricalDataClient(KEY, SECRET_KEY)
params = StockLatestQuoteRequest(symbol_or_symbols=df.index.to_list())
quotes = data_client.get_stock_latest_quote(params)

# add to dataframe
df["bid"] = [quotes[x].bid_price for x in df.index]
df["ask"] = [quotes[x].ask_price for x in df.index]
df.head()

,tradable,shortable,bid,ask
SPY,True,True,471.1,471.23
AAPL,True,True,194.8,194.82
IBM,True,True,0.0,162.02


### Submitting Orders

In [40]:
order = MarketOrderRequest(
    symbol="SPY",
    qty=10,
    side=OrderSide.BUY,
    time_in_force=TimeInForce.DAY
    )
_ = trading_client.submit_order(order)

In [41]:
order = MarketOrderRequest(
    symbol="AAPL",
    qty=5,
    side=OrderSide.SELL,
    time_in_force=TimeInForce.DAY
    )
_ = trading_client.submit_order(order)

### Get open positions

In [42]:
# List of all positions
positions = trading_client.get_all_positions()

# Print the quantity of shares for each position.
for position in positions:
    print("{} shares of {}".format(position.qty, position.symbol))

-5 shares of AAPL
10 shares of SPY


In [14]:
# Dataframe of positions
positions = trading_client.get_all_positions()

if len(positions) > 0:
    symbols       = [x.symbol for x in positions]
    qty           = [float(x.qty) for x in positions]
    market_value  = [float(x.market_value) for x in positions]
    cost_basis    = [float(x.cost_basis) for x in positions]

    df = pd.DataFrame(list(zip(qty, market_value, cost_basis)), columns=['qty','mktval','cost_basis'], index=symbols)

else:
    print('No positions')

df.head()

,qty,mktval,cost_basis
AAPL,-5.0,-974.150,-974.0
SPY,10.0,4713.199,4712.2


In [15]:
# Add cash balance to show balance sheet
account = trading_client.get_account()
df.loc['cash','mktval'] = float(account.cash)
df.head()

,qty,mktval,cost_basis
AAPL,-5.0,-974.150,-974.0
SPY,10.0,4713.199,4712.2
cash,NaN,96265.840,NaN


In [16]:
print(f'Portfolio value (from positions):\t${df.mktval.sum():,.2f}')
print(f'Portfolio value (from account):\t\t${float(account.portfolio_value):,.2f}')

Portfolio value (from positions):	$100,004.89
Portfolio value (from account):		$100,004.98


### Undo the trades

In [33]:
order = MarketOrderRequest(
    symbol="SPY",
    qty=10,
    side=OrderSide.SELL,
    time_in_force=TimeInForce.DAY
    )
_ = trading_client.submit_order(order)

In [34]:
order = MarketOrderRequest(
    symbol="AAPL",
    qty=5,
    side=OrderSide.BUY,
    time_in_force=TimeInForce.DAY
    )
_ = trading_client.submit_order(order)

In [38]:
positions = trading_client.get_all_positions()

# Print the quantity of shares for each position.
for position in positions:
    print("{} shares of {}".format(position.qty, position.symbol))

In [39]:
account = trading_client.get_account()
print(f'Portfolio value:\t${float(account.portfolio_value):,.2f}')

Portfolio value:	$100,002.61


### Close all positions

In [ ]:
trading_client.close_all_positions(cancel_orders=True)

In [45]:
positions = trading_client.get_all_positions()
for position in positions:
    print("{} shares of {}".format(position.qty, position.symbol))